In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df_producto=pd.read_csv('../data/impo_doc_total.csv',
                        dtype={'ncm':'str','CUIT':'str','empresa':'str','enmienda':'str','ncm_descri':'str','sim':'str','pais':'str','pais_descri':'str','dia':'str','mes':'str','anio':'str'})

In [3]:
# Correcciones de lectura
df_producto.pais_descri=df_producto.pais_descri.apply(lambda x: x.capitalize())
df_producto.empresa=df_producto.empresa.apply(lambda x: x.lower())

#Fob por tonelada importada
df_producto['fob_unitario_ton']=df_producto.fob/df_producto.pnet*1000

# #Crear columna que una ncm con su descri
# def recortar_descri_ncm(ncm_con_descri, caracteres=75):
#     '''Recortar la descripcion de las ncm, como para que en el dropdown aparezcan bien. Sino puede ser muy largo'''
#     if len(ncm_con_descri) > caracteres:
#         return ncm_con_descri[:caracteres-3]+'...'
#     else:
#         return ncm_con_descri
    
# df_producto['ncm_con_descri']=(df_producto.ncm.astype(str)+' - '+df_producto.ncm_descri).apply(recortar_descri_ncm)

# Fechas para usarlas en los graficos
desde=df_producto.sort_values(['anio','mes','dia'], ascending=True)['anio'][0]
hasta=df_producto.sort_values(['anio','mes','dia'], ascending=True)['anio'].iloc[-1]


inicio=df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).mes[0]+'/01/'+df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).anio[0]
mes=int(df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).mes.iloc[-1])+1
fin=str(mes)+'/01/'+df_producto.sort_values(['anio','mes'],ascending=True).reset_index(drop=True).anio.iloc[-1]
monthDates = pd.DataFrame({
    'fecha': pd.date_range(start=inicio, end=fin, freq='M').strftime('%m-%Y')
})

In [4]:
#Funcion para precio de referencia (endogeno)

def precio_ref(df):
    df_mensual=df.groupby(['mes','anio'],as_index=False).sum().sort_values(['anio','mes'])
    df_mensual['fob_unitario_ton_ref']=df_mensual.fob/df_mensual.pnet*1000
    df=pd.merge(left=df,right=df_mensual[['anio','mes','fob_unitario_ton_ref']], on=['anio','mes'],how='left')
    df['fob_unitario_ton_capit']=df.fob_unitario_ton*df.fob_unitario_ton_ref.iloc[-1]/df.fob_unitario_ton_ref
    df['diferencia_ref']=df.fob_unitario_ton-df.fob_unitario_ton_ref
    df['diferencia_ref_capi']=df.fob_unitario_ton_capit-df.fob_unitario_ton_ref.iloc[-1]
    return df

# Primer filtro está dado por la ncm

def ncm_filtro(df,ncm):
    df=df.loc[df.ncm==ncm].reset_index(drop=True)
    df['fecha']=df.mes+'-'+df.anio
    # df['fecha']=pd.to_datetime(df["fecha"]).dt.strftime('%m-%Y')
    df=df.sort_values(['anio','mes','dia'],ascending=True).reset_index(drop=True)
    return precio_ref(df)

def sim_unique(ncm):
    # return df.sim.unique()
    return df_producto[df_producto.ncm==ncm].sim.unique()

def recortar_descri(pais,largo=15):
    if len(pais)<=largo:
        return pais
    else: return str(pais[:largo]+'...')
    
df_producto.pais_descri.apply(recortar_descri)

0               China
1            Paraguay
2            Paraguay
3          Eslovaquia
4              Italia
              ...    
4242761        Brasil
4242762        Brasil
4242763        Brasil
4242764        Brasil
4242765        Brasil
Name: pais_descri, Length: 4242766, dtype: object

In [5]:
#Segundo filtro dado por el codigo sim. En caso de dafualt, no hacer filtro

def sim_filtro(df,ncm, sim='default', pais='default'):
    if sim=='default' and pais=='default':
        return ncm_filtro(df,ncm)
    elif sim=='default' and pais!='default':
        pais=pais.capitalize()
        return ncm_filtro(df,ncm)[ncm_filtro(df,ncm).pais_descri==pais].reset_index(drop=True)
    elif sim!='default' and pais=='default':
        return ncm_filtro(df,ncm)[ncm_filtro(df,ncm).sim==sim].reset_index(drop=True)
    else:
        pais=pais.capitalize()
        return ncm_filtro(df,ncm)[(ncm_filtro(df,ncm).sim==sim)&(ncm_filtro(df,ncm).pais_descri==pais)].reset_index(drop=True)

    
def get_top_paises(df, n_paises='default'):
    if n_paises=='default': return df
    else:
        top_df=df.groupby(['pais','pais_descri'],as_index=False).sum().sort_values('fob',ascending=False).pais_descri.unique()[:n_paises]
        return df[df.pais_descri.isin(top_df)].reset_index(drop=True)


In [6]:
def get_descri_nomen(df, palabras:int = 15):
    '''Entra lista de dataFrames y numero maximo de palabras'''
    for d in df.ncm_descri.unique():
        if type(d)!=float:
            if len(d.split(' '))>palabras:
                a=' '.join(d.split(' ')[:palabras])
                return a+'...'
            else: 
                return d

In [12]:
def precio_boxplot_capitalizado(df,ncm,sim='default', n_paises='default', pais='default'):
     ncm=str(ncm)
     df=sim_filtro(df,ncm,sim)
     df=get_top_paises(df,n_paises)
     producto=df.ncm_descri.unique()[0][:40]
     ncm=df.ncm.unique()[0]
     if sim=='default' and n_paises=='default':
          title_text=f'FOB por tonelada importada {desde}-{hasta} de: <br>"{get_descri_nomen(df,9)}"<br>NCM:{ncm}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales'     
     elif sim=='default' and n_paises!='default':
          title_text=f'FOB por tonelada importada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales, top {n_paises}'
     elif sim!='default' and n_paises=='default':
          title_text=f'FOB por tonelada importada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}-{sim}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales'
     elif sim!='default' and n_paises!='default':
          title_text=f'FOB por tonelada importada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}-{sim}<br> <sup> Capitalizado al último precio disponible. Precio de referencia a partir de datos mensuales, top {n_paises}'
            
     precio_soja_boxplot=px.box(
     df.round(1),
     # x='fecha',
     x='fob_unitario_ton_capit',
     # color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob_unitario_ton_capit':'Precio capitalizado',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'fob_unitario_ton_ref': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                 'fecha',
                    'fob_unitario_ton',
                    'fob_unitario_ton_capit',
                    'fob_unitario_ton_ref',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= '', 
                                   # range=[producto['fob_unitario_ton_capit'].min()-100,producto['fob_unitario_ton_capit'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='Fob unitario')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=400, width=750,
                                   template = 'none',
                                   title=dict(text=title_text,
                                              y=0.95),
                                   # title_text=title_text,
                                   showlegend=False,
                                   margin=dict(t=150))
     
     precio_soja_boxplot.add_vline(x=df.fob_unitario_ton_ref.iloc[-1],line_dash="dash",line_color="blue")

     return precio_soja_boxplot

def plot_precio(df,ncm,sim='default', n_paises='default', color='pais_descri', pais='default'):
     '''color=pais_descri o empresa'''
     ncm=str(ncm)
     df=sim_filtro(df,ncm,sim,pais)
     df=get_top_paises(df,n_paises)
     df.pais_descri=df.pais_descri.apply(recortar_descri)
     df.empresa=df.empresa.apply(recortar_descri)
     producto=df.ncm_descri.unique()[0][:40]
     ncm=df.ncm.unique()[0]
     if sim=='default' and n_paises=='default':
          title_text=f'FOB por tonelada importada {desde}-{hasta} de: <br>"{get_descri_nomen(df,9)}"<br>NCM:{ncm}<br> <sup> Precio de referencia a partir de datos mensuales'     
     elif sim=='default' and n_paises!='default':
          title_text=f'FOB por tonelada importada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}<br> <sup> Precio de referencia a partir de datos mensuales, top {n_paises} paises'
     elif sim!='default' and n_paises=='default':
          title_text=f'FOB por tonelada importada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}-{sim}<br> <sup> Precio de referencia a partir de datos mensuales'
     elif sim!='default' and n_paises!='default':
          title_text=f'FOB por tonelada importada {desde}-{hasta} de: <br>"{get_descri_nomen(df,12)}"<br>NCM:{ncm}-{sim}<br> <sup> Precio de referencia a partir de datos mensuales, top {n_paises} paises'
                 
     precio_soja_plot=px.scatter(
     df.round(1),
     x='fecha',
     y='diferencia_ref',
     color=color,
     category_orders={"fecha": monthDates.fecha,
                      'empresa': df.empresa.sort_values(ascending=True),
                           },
     labels={
          "pais_descri": "Origen",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'fob_unitario_ton_ref': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'fob_unitario_ton_ref',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_plot.update_yaxes(title_text= 'Diferencia en USD', 
                                   range=[df['diferencia_ref'].min()-100,df['diferencia_ref'].max()+100]
                                   )
     precio_soja_plot.update_xaxes(title_text='',type='category')

     precio_soja_plot.update_traces(marker=dict(size=12,
                                   line=dict(width=2,
                                             color='DarkSlateGrey')),
                    selector=dict(mode='markers'))

     precio_soja_plot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=700, width=900,
                                   template = 'none',
                                   title=dict(text=title_text,
                                              y=0.95),
                                   # title_text=title_text,
                                   # showlegend=False,
                                   margin=dict(t=150))
                                   

     return precio_soja_plot


In [8]:
sim_filtro(df_producto,'84798999').empresa.unique()

array(['volkswagen argentina s a', 'biggi santiago fernando',
       'mazzarella hernan ariel', ..., 'harinas bajo hondo s a c i',
       'faurecia sistemas de escape argentina s.a.', 'macroplast sa'],
      dtype=object)

In [13]:
ncm='84733043'
# precio_boxplot_capitalizado(df=df_producto,
#                             ncm=ncm,
#                             # sim=sim_unique(ncm)[1],
#                             )

plot_precio(df=df_producto,
            ncm=ncm,
            # sim=sim_unique(ncm)[0],
            color='empresa',
            pais='China'
            )

In [10]:
sim_unique(ncm='12019000')

array(['190C', '110B'], dtype=object)

In [14]:
plot_precio(df=df_producto, ncm='12019000',
            # sim='190C',
            color='empresa')

In [108]:
plot_precio(df=df_producto, ncm='12019000',color='empresa').write_html('../output/diferencia con ref soja impo empresa.html')
precio_boxplot_capitalizado(df_producto,'12019000').write_html('../output/impo soja boxplot.html')

In [106]:
writer = pd.ExcelWriter(f'../output/importaciones selecionadas.xlsx', engine='xlsxwriter')
sim_filtro(df_producto,'12019000').to_excel(writer, sheet_name='soja', index=False)
# tabla_1('trigo').to_excel(writer, sheet_name='trigo', index=False)
# tabla_1('maiz').to_excel(writer, sheet_name='maiz', index=False)
writer.save()
